In [1]:
%matplotlib inline

In [3]:
import argparse
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

In [4]:
import music21
import os
os.environ["musescoreDirectPNGPath"] = "/Applications/MuseScore\ 3/bin/MuseScore.exe"
os.environ["musicxmlPath"] = "/Applications/MuseScore\ 3/bin/MuseScore.exe"
song = music21.converter.parse('bach/chor002.midi')

from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import torchvision.transforms as transforms
import torch
import torchvision.utils as vutils

#import necessary modules, and set path to use musescore

In [296]:
#split images according to criterion specified
#i.e. four quarter notes per image
#then save to file 
songs = []
for i in range(1, 111):
    if len(str(i))==1:
        e = "00" + str(i)
    elif len(str(i))==2:
        e = "0" + str(i)
    else:
        e = str(i)
    song = music21.converter.parse('Downloads/chor{}.mid'.format(e))
    
    for voice in range(len(song)):
        prev = 0
        for note in range(len(song[voice].notes)):
            if song[voice].notes.stream()[:note].highestOffset%3==0 and \
            song[voice].notes.stream()[:note].highestOffset>0:
                s1 = music21.stream.Stream()
                for k in range(prev, note):
                    s1.append(song[voice].notes.stream()[k])
                prev = note
                tmp_path = s1.write(fmt = 'musicxml.png')
                #write to permanent file, 
                #a hack bc music21's path specifying feature does not work for me
                in_ = open(tmp_path, "rb")
                with open("bach/img{}.png".format(e+str(voice)+str(note)), "wb")\
                as file:
                    file.write(in_.read())

        in_.close()
                          
    
#         break
        
#         song.flat.notes.stream()[:10].show()

In [61]:
#finally, the png images have "transparencies"
#which need to be changed into white pixels 
#(a gap in the PIL functionality that PyTorch uses)
from PIL import Image
import re

for i in os.listdir("bach"):
    if re.match("img\d+.png", i) != None:
        # Load the image and make into Numpy array
        rgba = np.array(Image.open("bach/" + i))

        # Make image transparent white anywhere it is transparent
        rgba[rgba[...,-1]==0] = [255,255,255,0]

        # Make back into PIL Image and save
        Image.fromarray(rgba).save("bach/" + i)

In [62]:
#making labels for cgan
songs = []
file = open("labels.txt", "a+")
t = 0
for i in range(78, 79):
    if len(str(i))==1:
        e = "00" + str(i)
    elif len(str(i))==2:
        e = "0" + str(i)
    else:
        e = str(i)
    song = music21.converter.parse('bach2/chor{}.midi'.format(e))
    
    for voice in range(len(song)):
        prev = 0
        for note in range(len(song[voice].notes)):
            if song[voice].notes.stream()[:note].highestOffset%3==0 and\
            song[voice].notes.stream()[:note].highestOffset>0:
                s1 = music21.stream.Stream()
                for k in range(prev, note):
                    s1.append(song[voice].notes.stream()[k])
                if str(song[voice].notes.stream()[prev])[-2]=="#" \
                or str(song[voice].notes.stream()[prev])[-2]=="-":
                    file.write(e+str(voice)+str(note) + "\t" + \
                               str(song[voice].notes.stream()[prev])[-3:-1])
                else:
                    file.write(e+str(voice)+str(note) + "\t" + \
                               str(song[voice].notes.stream()[prev])[-2])
                file.write("\n")
                t+=1
#                 print(str(song[voice].notes.stream()[prev])[-2])
                prev = note
#                 tmp_path = s1.write(fmt = 'musicxml.png')
                

file.close()

#         in_.close()
                          
    
#         break
        
#         song.flat.notes.stream()[:10].show()